In [5]:
 # -- coding: utf-8 -- 


뉴스 헤드라인을 통해 뉴스의 카테고리를 분류하는 모델을 구성한다.


1. 분류: 정치, 사회, 경제, 세계, IT, 생활
2. 방법
    - 다음, 네이버 뉴스의 헤드라인 크롤링 (06월 ~ 12월) (중복제거 전 2231070: -> 중복제거됨: 1579514)
    - 중복 제거 전
            foreign     543310
            society     522570
            economic    438658
            politics    431247
            culture     193063
            digital     102222
            label         1103
            2232173
            
    - 중복 제거 후
            foreign     191344
            society     391471
            economic    396014
            politics    362782
            culture     150445
            digital      87451
            label            7
            1579514
            
    - 레이블에 따라 주요 키워드를 선별한다.
        - 주요키워드: 분류를 구분할 수 있을만한 단어들.
        - 특정 카테고리당 단어가 많이 나온 횟수 측정
            - 카테고리 상관없이 한번 추출하고
            - 카테고리 있게 추출해서        
            - 그 단어가 유의미한 구분을 할 수 있게 도와주는지 측정.
            - (선택1) 특정 시간대에 단어가 다수 사용되어 예측을 떨어뜨리는 것에 대한 방안
        - 키워드당 분포를 노멀라이즈해서 어느정도 유니크한것만 사용하는 걸루
            
    - 50개(or threshold: 20%)를 선별하여 하나 하나를 column으로 만듬.
    - 문장을 주요키워드의 유무에 따라 카테고리를 선별하도록 구성.
    - input은 keyword의 개수, output은 카테고리의 개수
    
3. 의문
    - 특정 기간에만 언급되는 뉴스들은 어떻게? (선택1)
    

In [128]:
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')


data1 = pd.read_csv('./data/Data06.csv')
data1['date'] = 201806
data = data1.copy()
data2 = pd.read_csv('./data/Data07.csv')
data2['date'] = 201807
data = data.append(data2)
data3 = pd.read_csv('./data/Data08.csv')
data3['date'] = 201808
data = data.append(data3)
data4 = pd.read_csv('./data/Data09.csv')
data4['date'] = 201809
data = data.append(data4)
data5 = pd.read_csv('./data/Data10.csv')
data5['date'] = 201810
data = data.append(data5)
data6 = pd.read_csv('./data/Data11.csv')
data6['date'] = 201811
data = data.append(data6)
data7 = pd.read_csv('./data/Data12.csv')
data7['date'] = 201812
data = data.append(data7)
data = data.reset_index()
data.tail()

,index,label,subject,time,url,date
1579509,92966,digital,"애플 아이폰, 중국서 판매금지 당했다",8:40,https://media.daum.net/breakingnews/digital?pa...,201812
1579510,92967,digital,"노웅래 의원, '통신재난' 피해보상 기준·절차 마련 추진",8:36,https://media.daum.net/breakingnews/digital?pa...,201812
1579511,92968,digital,"""듣고 싶은 것만 듣는다""..SK텔레콤, 신규 음악 플랫폼 'FLO' 출시",8:36,https://media.daum.net/breakingnews/digital?pa...,201812
1579512,92969,digital,"""내 취향 음악만 듣는다""..SKT, 새 음악서비스 '플로(FLO)' 출시",8:35,https://media.daum.net/breakingnews/digital?pa...,201812
1579513,92970,digital,"업비트, 국제보안인증 3종 획득..암호화폐 거래소중 처음",8:34,https://media.daum.net/breakingnews/digital?pa...,201812


# Data Preprocessing

In [129]:
print(data.shape[0])

1579514


# Shuffle

In [462]:
data = sklearn.utils.shuffle(data)
data = data.reset_index()
data = data.drop('index', axis=1)
data.head()

,level_0,label,subject,time,url,date
0,1173918,culture,2018 서울아리랑페스티벌 열려,22:00,https://media.daum.net/breakingnews/culture?pa...,201810
1,986706,politics,한복입고 공연 관람,19:10,https://media.daum.net/breakingnews/politics?p...,201810
2,1098591,economic,"수제맥주 강화하는 신세계푸드, 여의도 IFC몰에 데블스도어 6호점",14:39,https://media.daum.net/breakingnews/economic?p...,201810
3,439211,culture,'이우환 위작설 무마시도' 전 수사관 2심서 징역 1년2개월,11:11,https://media.daum.net/breakingnews/culture?pa...,201807
4,1413035,society,전국책읽는도시협의회 초대 회장에 김승수 전주시장,15:55,https://media.daum.net/breakingnews/society?pa...,201811


In [463]:
divider = int(data.shape[0] * 0.2)
test =  data[0:divider].copy()
train =  data[divider+1:].copy()

print("Total: ", data.shape[0])
print("TestSet: ",len(test))
print("TrainSet: " ,len(train))

Total:  1579514
TestSet:  315902
TrainSet:  1263611


## 형태소 분리 테스트

In [365]:
def add_row(df, row):
    df.loc[-1] = row
    df.index = df.index + 1  
    return df.sort_index()

In [367]:
import khaiii as kh

In [487]:
def extractKeyword(data_set, keyword_set):
    api = kh.KhaiiiApi()
    api.open()
    
    max_size = len(data_set)

    for i  in range(len(data_set)):
        if (i % 1000 == 0):
            print(str(i / max_size * 100) + "%...")
        
        sentenceRaw = data_set.loc[i]
        # 카테고리
        category = sentenceRaw['label']
        
        # 형태소 분석을 진행할 헤드라인
        sentence = sentenceRaw['subject']
    
        # 필요없는 부분 삭제 (e.g. [포토])
        subTitleEndPos = sentence.find(']')
        if subTitleEndPos != -1:
            sentence = sentence[subTitleEndPos + 1:]
        
        if (len(sentence) > 0):
#             print(sentence)
            for word in api.analyze(sentence):
                for m in word.morphs:
                    # NNG 일반명사, NNP 고유명사, NNB 의존명사
                    # 명사거나 2글자 이상
                    if 'NN' in m.tag and len(m.lex) > 1:
                        r = priority_keyword.loc[priority_keyword['keyword'] == m.lex]

                        # 존재하지 않을경우
                        if len(r) == 0:
                            add_row(priority_keyword, [m.lex, 0, 0, 0, 0, 0, 0, 0])

                        priority_keyword.loc[priority_keyword['keyword'] == m.lex, 'frequency'] = priority_keyword.loc[priority_keyword['keyword'] == m.lex, 'frequency'] + 1
                        priority_keyword.loc[priority_keyword['keyword'] == m.lex, category] = priority_keyword.loc[priority_keyword['keyword'] == m.lex, category] + 1

In [464]:
train = train.reset_index()

In [498]:
# 키워드 보관용 DafaFrame,  실행할때마다 초기화 되어야하므로 extractKeyword와 같이 배치!
keywords_cols = ['keyword', 'frequency', 'foreign', 'society', 'economic', 'politics', 'culture', 'digital']
priority_keyword = pd.DataFrame(columns=keywords_cols)

target = train[0:30000]
extractKeyword(target, prioty_keyword)

0.0%...
3.3333333333333335%...
6.666666666666667%...
10.0%...
13.333333333333334%...
16.666666666666664%...
20.0%...
23.333333333333332%...
26.666666666666668%...
30.0%...
33.33333333333333%...
36.666666666666664%...
40.0%...
43.333333333333336%...
46.666666666666664%...
50.0%...
53.333333333333336%...
56.666666666666664%...
60.0%...
63.33333333333333%...
66.66666666666666%...
70.0%...
73.33333333333333%...
76.66666666666667%...
80.0%...
83.33333333333334%...
86.66666666666667%...
90.0%...
93.33333333333333%...
96.66666666666667%...


In [499]:
priority_keyword.copy().sort_values('frequency', ascending=False).head(10)

,keyword,frequency,foreign,society,economic,politics,culture,digital
27935,대통령,777,86,36,18,625,10,2
27783,대표,558,29,61,98,329,16,25
27430,개최,514,6,157,146,92,67,46
27744,시장,470,13,116,165,117,20,39
27131,상승,443,7,10,413,6,2,5
28096,한국,420,44,65,145,84,53,29
27818,대비,418,9,33,343,21,6,6
27398,장관,403,23,45,68,255,5,7
28083,트럼프,373,280,6,16,69,1,1
28080,회담,372,86,13,8,263,2,0


In [500]:
copied = priority_keyword.copy()
print("정치 카테고리와 가장 연관된 키워드는 {} 입니다.".format(copied.sort_values('politics', ascending=False).head(1)['keyword'].values[0]))
print("경제 카테고리와 가장 연관된 키워드는 {} 입니다.".format(copied.sort_values('economic', ascending=False).head(1)['keyword'].values[0]))
print("사회 카테고리와 가장 연관된 키워드는 {} 입니다.".format(copied.sort_values('society', ascending=False).head(1)['keyword'].values[0]))
print("세계 카테고리와 가장 연관된 키워드는 {} 입니다.".format(copied.sort_values('foreign', ascending=False).head(1)['keyword'].values[0]))
print("생활/문화 카테고리와 가장 연관된 키워드는 {} 입니다.".format(copied.sort_values('culture', ascending=False).head(1)['keyword'].values[0]))
print("IT 카테고리와 가장 연관된 키워드는 {} 입니다.".format(copied.sort_values('digital', ascending=False).head(1)['keyword'].values[0]))

정치 카테고리와 가장 연관된 키워드는 대통령 입니다.
경제 카테고리와 가장 연관된 키워드는 상승 입니다.
사회 카테고리와 가장 연관된 키워드는 경찰 입니다.
세계 카테고리와 가장 연관된 키워드는 트럼프 입니다.
생활/문화 카테고리와 가장 연관된 키워드는 개최 입니다.
IT 카테고리와 가장 연관된 키워드는 출시 입니다.


In [501]:
priority_keyword.copy().sort_values('foreign', ascending=False).head(10)

,keyword,frequency,foreign,society,economic,politics,culture,digital
28083,트럼프,373,280,6,16,69,1,1
27652,중국,272,131,21,60,26,20,14
27599,미국,184,102,3,46,27,3,3
28080,회담,372,86,13,8,263,2,0
27935,대통령,777,86,36,18,625,10,2
27895,사망,173,74,79,6,6,5,3
27993,무역,151,70,5,60,11,3,2
28081,정상,318,67,23,22,196,7,3
27915,종합,359,55,71,96,125,10,2
27732,일본,144,55,21,33,14,12,9
